<a href="https://colab.research.google.com/github/Pushpalatha-H/DL/blob/main/7_Neural_Network_From_Sctrach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Implement Neural Network (or Logistic Regression) From Scratch**

**Predict if a person buy life insurance based on age using Logistic Regression**

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/codebasics/py/master/DeepLearningML/7_nn_from_scratch/insurance_data.csv")

In [3]:
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


**Split into train and test**

In [4]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df[['age', 'affordibility']], df.bought_insurance)


In [5]:
len(x_train)

21

In [6]:
len(x_test)

7

**Scale the data so both age and affordibility are in same rage**

In [7]:
# scale down
x_train_scaled = x_train.copy()
x_train_scaled['age'] = x_train_scaled['age'] / 100

x_test_scaled = x_test.copy()
x_test_scaled['age'] = x_test_scaled['age'] / 100

**Model building**

In [8]:
model = keras.Sequential([
    keras.layers.Dense(1, input_shape=(2,), activation = 'sigmoid',kernel_initializer='ones', bias_initializer='zeros' )
])
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [9]:
model.fit(x_train_scaled, y_train)

1/1 [==============================] - 1s 1s/step - loss: 0.7101 - accuracy: 0.5238


In [10]:
model.evaluate(x_test_scaled, y_test)

1/1 [==============================] - 0s 322ms/step - loss: 0.7330 - accuracy: 0.4286


[0.7329796552658081, 0.4285714328289032]

In [11]:
model.predict(x_test_scaled)

1/1 [==============================] - 0s 136ms/step


array([[0.5643265 ],
       [0.81115615],
       [0.8126817 ],
       [0.6337759 ],
       [0.8259857 ],
       [0.5470628 ],
       [0.6452902 ]], dtype=float32)

In [12]:
# w1, w2, bias
coef, intercept = model.get_weights()
coef, intercept

(array([[0.9990001],
        [0.999    ]], dtype=float32),
 array([-0.00099998], dtype=float32))

**Now we got w1: 0.9990001, w2: 0.999, bias : -0.00099998**

**get w1, w2, bias with out the model**

In [13]:
# get w1, w2, bias with out the model
import math
def sigmoid(x):
  return 1/(1+math.exp(-x))
sigmoid(23)

0.9999999998973812

In [14]:
def prediction_function(age, affordibility):
  weighted_sum = coef[0]*age + coef[1]*affordibility + intercept
  return sigmoid(weighted_sum)

In [15]:
prediction_function(.23, 1)

<ipython-input-13-bca6ae5a073e>:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 1/(1+math.exp(-x))


0.7734280495945837

**Start implementing gradient descent in plain python. Again the goal is to come up with same w1, w2, bias that kernel model calculated.**

**Fist write couple of helper routines such as sigmoid and log_loss**

In [24]:
def sigmoid_numpy(x):
  return 1 / (1+np.exp(-x))
sigmoid_numpy(np.array([12,0,1]))

array([0.99999386, 0.5       , 0.73105858])

In [25]:
def log_loss(y_true, y_predicted):
  epsilon = 1e-15
  y_predicted_new = [max(i, epsilon) for i in y_predicted]
  y_predicted_new = [min(i, epsilon) for i in y_predicted_new]
  y_predicted_new = np.array(y_predicted_new)
  return -np.mean(y_true* np.log(y_predicted_new) + (1 - y_true) * np.log(1 - y_predicted_new))

**Implement own custom neural network class**

In [26]:
class myNN:
  def __init__(self):
    self.w1 = 1
    self.w2 = 1
    self.bias = 0

  def fit(self, x, y, epochs, loss_threshold):
    self.w1, self.w2, self.w3 = self.gradient_descent(x['age'], x['affordibility'], y, epochs, loss_threshold)
    print(f'Final weight and bias:, w1:{self.w1}, w2:{self.w2}, w3:{self.w3}')

  def predict(self, x_test):
    weighted_sum = self.w1*x_test['age'] + self.w2*x_test['affordibility'] + self.bias
    return sigmoid_numpy(weighted_sum)

  def gradient_descent(self, age, affordibility,y_true, epochs, loss_threshold):
    w1 = 1
    w2 = 1
    b = 0
    rate = 0.1
    n = len(age)

    for i in range(epochs):
      weighted_sum = w1*age + w2 * affordibility + b
      y_predicted = sigmoid_numpy(weighted_sum)
      loss = log_loss(y_true , y_predicted)

      w1d = 1/n *np.dot(np.transpose(age), (y_predicted - y_true))
      w2d = 1/n *np.dot(np.transpose(affordibility), (y_predicted - y_true))
      bd = 1/n * np.mean(y_predicted - y_true)

      w1 = w1 - rate * w1d
      w2 = w2 - rate * w2d
      b = b - rate * bd

      if i%50==0:
        print(f'Epochs:{i}, w1:{w1}, w2:{w2}, bias:{b}, loss:{loss}')

      if loss<=loss_threshold:
        print(f'Epochs:{i}, w1:{w1}, w2:{w2}, bias:{b}, loss:{loss}')
        break

    return w1, w2, b

In [27]:
customModel = myNN()
customModel.fit(x_train_scaled, y_train, epochs = 10, loss_threshold = 0.4)

Epochs:0, w1:0.9964149011697049, w2:0.9865157919330216, bias:-0.0010708103174710268, loss:18.091740016381785
Final weight and bias:, w1:0.9677310775983496, w2:0.8737171582404895, w3:-0.01027437983348023


In [28]:
coef, intercept

(array([[0.9990001],
        [0.999    ]], dtype=float32),
 array([-0.00099998], dtype=float32))

**This shows that in the end we were able to come up with same value of w1,w2 and bias using a plain python implementation of gradient descent function**

In [29]:
x_test_scaled

,age,affordibility
21,0.26,0
27,0.46,1
2,0.47,1
6,0.55,0
5,0.56,1
18,0.19,0
7,0.60,0


**1. Prediction using custom model**

In [33]:
customModel.predict(x_test_scaled)

21    0.562573
27    0.788994
2     0.790601
6     0.630008
5     0.804655
18    0.545838
7     0.641214
dtype: float64

**2. Prediction using tensorflow model**

In [34]:
model.predict(x_test_scaled)

1/1 [==============================] - 0s 45ms/step


array([[0.5643265 ],
       [0.81115615],
       [0.8126817 ],
       [0.6337759 ],
       [0.8259857 ],
       [0.5470628 ],
       [0.6452902 ]], dtype=float32)

**Above you can compare predictions from our own custom model and tensoflow model. You will notice that predictions are almost same**